In [9]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm


df = pd.read_excel('台塑企業_ 產品寶典20210303.xlsx',engine='openpyxl')
display(df.head(3))
display(df[df['品名']=='MA'])

產品集合 = set(df['品名'].values)

'''
train_df = pd.read_csv('Train_Product_Data_2021_0114.csv',index_col=0).drop(['Forward','Backward','label_for_train'],axis=1)
train_df = train_df.dropna(axis=0)
display(train_df.head(3))

val_df = pd.read_csv('Val_Product_Data_2021_0114.csv',index_col=0).drop(['Forward','Backward'],axis=1)
val_df = val_df.dropna(axis=0)
display(val_df.head(3))
'''

,RIGID,分機,公司代號,公司事業部門,品名,新增
0,黃淑玲,7190,11,台塑塑膠部,CAUSTIC SODA LIQUID FLAKE PEARLS MICROPEARLS,NaN
1,黃淑玲,7190,11,台塑塑膠部,CAUSTIC SODA LIQUID,NaN
2,黃淑玲,7190,11,台塑塑膠部,COMMODITY FORMOSACN,NaN


,RIGID,分機,公司代號,公司事業部門,品名,新增
531,NaN,NaN,2P,南亞塑四部化學品部,MA,NaN


"\ntrain_df = pd.read_csv('Train_Product_Data_2021_0114.csv',index_col=0).drop(['Forward','Backward','label_for_train'],axis=1)\ntrain_df = train_df.dropna(axis=0)\ndisplay(train_df.head(3))\n\nval_df = pd.read_csv('Val_Product_Data_2021_0114.csv',index_col=0).drop(['Forward','Backward'],axis=1)\nval_df = val_df.dropna(axis=0)\ndisplay(val_df.head(3))\n"

In [10]:
val_df = pd.read_csv('preprocess_for_SQUAD_wordninja.csv',index_col=0)[['45A','Y_label']]
val_df.columns = ['string_X_train','Y_label']
val_df

,string_X_train,Y_label
0,MASS PVC RESIN B-57 QUANTITY 175 MT AT 1300 US...,MASS PVC RESIN B-57
1,PHTHALIC ANHYDRIDE PA QUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE
2,COMMODITY LLDPE TAISOX 3470 QUANTITY 320 MT 2 ...,LLDPE TAISOX
5,ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,HDPE TAISOX 8010
6,ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,HDPE TAISOX 8010
...,...,...
3609,COMMODITY MONO ETHYLENE GLYCOL x 000 D QUANTIT...,MONO ETHYLENE GLYCOL
3610,CFR KOBE JAPAN x 000 D VISCOSE RAYON STAPLE FI...,VISCOSE RAYON STAPLE FIBER
3611,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN
3612,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN


In [11]:
train_df = val_df

# 如果品名是單詞的話 前後加個空白

In [12]:
for i in df.index:
    if ' ' not in df.loc[i,'品名']:
        name = df.loc[i,'品名']
        df.loc[i,'品名'] = f' {name} '
        assert df.loc[i,'品名'][0] == ' '
        assert df.loc[i,'品名'][-1] == ' '

In [13]:
display(df[df['品名']=='MA'])
display(df[df['品名']==' MA '])

,RIGID,分機,公司代號,公司事業部門,品名,新增


,RIGID,分機,公司代號,公司事業部門,品名,新增
531,NaN,NaN,2P,南亞塑四部化學品部,MA,NaN


In [14]:
產品集合 = set(df['品名'].values)

# find_fail_sample and drop fail_sample

In [15]:
def find_fail_sample(df):
    fails = []
    for i in df.index:
        context = df.loc[i,'string_X_train']
        answer = df.loc[i,'Y_label']
        if str(answer) not in str(context):
            fails.append(i)
    return fails
train_fails = find_fail_sample(train_df)
val_fails = find_fail_sample(val_df)
print(train_fails,val_fails)
display(val_df.loc[val_fails])
print(val_df.shape)
val_df = val_df.drop(val_fails,axis=0)
print(val_df.shape)

[182, 521, 745, 748, 758, 759, 760, 829, 832, 833, 875, 876, 891, 908, 920, 923, 925, 957, 987, 998, 1003, 1004, 1007, 1009, 1046, 1048, 1083, 1132, 1135, 1136, 1154, 1203, 1209, 1266, 1301, 1315, 1317, 1326, 1368, 1398, 1481, 1501, 1502, 1512, 1552, 1553, 1589, 1602, 1643, 1650, 1677, 1679, 1680, 1713, 1714, 1716, 1718, 1721, 1754, 1755, 1780, 1831, 1871, 1872, 1962, 1964, 1965, 2067, 2208, 2210, 2218, 2229, 2233, 2246, 2282, 2353, 2387, 2389, 2393, 2478, 2485, 2502, 2541, 2553, 2573, 2574, 2596, 2600, 2608, 2680, 2709, 2720, 2768, 2826, 2828, 2833, 2838, 2839, 2840, 2841, 2844, 2847, 2912, 2962, 2985, 2986, 3000, 3024, 3026, 3083, 3084, 3133, 3162, 3166, 3194, 3215, 3216, 3225, 3235, 3248, 3295, 3296, 3339, 3393, 3394, 3399, 3554, 3584, 3587, 3590, 3591, 3595, 3605, 3608] [182, 521, 745, 748, 758, 759, 760, 829, 832, 833, 875, 876, 891, 908, 920, 923, 925, 957, 987, 998, 1003, 1004, 1007, 1009, 1046, 1048, 1083, 1132, 1135, 1136, 1154, 1203, 1209, 1266, 1301, 1315, 1317, 1326, 1368, 

,string_X_train,Y_label
182,PHTHALIC ANHYDRIDE 504 MT AT USD 1010 M Tx 000...,NaN
521,PHTHALIC ANHYDRIDE 504 MT AT USD 980 M Tx 000 ...,NaN
745,9000 MT PHTHALIC ANHYDRIDE PAx 000 DAT USD 825...,NaN
748,9000 MT PHTHALIC ANHYDRIDE PAx 000 DAT USD 825...,NaN
758,9000 MT PHTHALIC ANHYDRIDE PAx 000 DAT USD 825...,NaN
...,...,...
3590,3600 MT 2 F CL x 000 D PHTHALIC ANHYDRIDE x 00...,NaN
3591,PHTHALIC ANHYDRIDE PAx 000 D QT Y 54 M TSx 000...,NaN
3595,108 MT 6 F CL OF PHTHALIC ANHYDRIDE AT 98000 P...,NaN
3605,108 MT 6 F CL OF PHTHALIC ANHYDRIDE AT USD 960...,NaN


(3198, 2)
(3064, 2)


In [16]:
def Collection_method(df,產品集合):
    labels = {}
    for i in tqdm(df.index):
        products = []
        for p in 產品集合:
            if p in df.loc[i,'string_X_train']:
                products.append(p)
        labels[i] = products
    predict = pd.DataFrame(index=labels.keys(),columns=['predict'])
    predict['predict'] = labels.values()
    return predict
predict = Collection_method(val_df,產品集合)
result = val_df.join(predict)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/3064 [00:00<?, ?it/s]

In [17]:
result

,string_X_train,Y_label,predict
0,MASS PVC RESIN B-57 QUANTITY 175 MT AT 1300 US...,MASS PVC RESIN B-57,"[ RESIN , PVC RESIN B-57, PVC RESIN]"
1,PHTHALIC ANHYDRIDE PA QUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE,"[PHTHALIC ANHYDRIDE PA, PA , PHTHALIC ANHYDRI..."
2,COMMODITY LLDPE TAISOX 3470 QUANTITY 320 MT 2 ...,LLDPE TAISOX,"[ TAISOX , LLDPE TAISOX]"
5,ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,HDPE TAISOX 8010,"[ HDPE , HDPE TAISOX 8010, TAISOX , HDPE TAISOX]"
6,ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,HDPE TAISOX 8010,"[ HDPE , HDPE TAISOX 8010, TAISOX , HDPE TAISOX]"
...,...,...,...
3609,COMMODITY MONO ETHYLENE GLYCOL x 000 D QUANTIT...,MONO ETHYLENE GLYCOL,"[MONO ETHYLENE GLYCOL, ETHYLENE ]"
3610,CFR KOBE JAPAN x 000 D VISCOSE RAYON STAPLE FI...,VISCOSE RAYON STAPLE FIBER,"[VISCOSE RAYON STAPLE FIBER, RAYON , RAYON ST..."
3611,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,"[ RESIN , PC RESIN, INA ]"
3612,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,"[ RESIN , PC RESIN, INA ]"


In [19]:
result.loc[2]

string_X_train    COMMODITY LLDPE TAISOX 3470 QUANTITY 320 MT 2 ...
Y_label                                                LLDPE TAISOX
predict                                    [ TAISOX , LLDPE TAISOX]
Name: 2, dtype: object

In [20]:
def get_acc(df):
    correct = []
    correct_label = []
    for i in df.index:
        if df.loc[i,'Y_label'] in df.loc[i,'predict']:
            correct.append('yes')
        else:
            correct.append('no')
    return correct

In [21]:
correct = get_acc(result)
result['correct'] = correct
result

,string_X_train,Y_label,predict,correct
0,MASS PVC RESIN B-57 QUANTITY 175 MT AT 1300 US...,MASS PVC RESIN B-57,"[ RESIN , PVC RESIN B-57, PVC RESIN]",no
1,PHTHALIC ANHYDRIDE PA QUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE,"[PHTHALIC ANHYDRIDE PA, PA , PHTHALIC ANHYDRI...",no
2,COMMODITY LLDPE TAISOX 3470 QUANTITY 320 MT 2 ...,LLDPE TAISOX,"[ TAISOX , LLDPE TAISOX]",yes
5,ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,HDPE TAISOX 8010,"[ HDPE , HDPE TAISOX 8010, TAISOX , HDPE TAISOX]",yes
6,ITEM 1 HDPE TAISOX 8010 200 MT USD 1100 MT CON...,HDPE TAISOX 8010,"[ HDPE , HDPE TAISOX 8010, TAISOX , HDPE TAISOX]",yes
...,...,...,...,...
3609,COMMODITY MONO ETHYLENE GLYCOL x 000 D QUANTIT...,MONO ETHYLENE GLYCOL,"[MONO ETHYLENE GLYCOL, ETHYLENE ]",yes
3610,CFR KOBE JAPAN x 000 D VISCOSE RAYON STAPLE FI...,VISCOSE RAYON STAPLE FIBER,"[VISCOSE RAYON STAPLE FIBER, RAYON , RAYON ST...",yes
3611,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,"[ RESIN , PC RESIN, INA ]",yes
3612,DESCRIPTION OF GOODS QUANTITY MT UNIT PRICE US...,PC RESIN,"[ RESIN , PC RESIN, INA ]",yes


In [22]:
result['correct'].value_counts()

yes    2003
no     1061
Name: correct, dtype: int64

In [23]:
result['correct'].value_counts()['yes']/len(val_df)

0.6537206266318538